# Lab6 决策树分类器

- 姓名：于洋淼
- 学号：2113644
- 专业：物联网工程

In [1]:
import numpy as np 
import pandas as pd
import math
import copy
from collections import Counter

### 导入数据

In [2]:
def read_data(name):
    df = pd.read_csv(name, encoding = 'gb2312')
    temp = np.array(df).tolist()
    for i in temp: # 第一列是编号，不需要
        i.pop(0)
    return temp

train1 = read_data("Watermelon-train1.csv")
train2 = read_data("Watermelon-train2.csv")
test1 = read_data("Watermelon-test1.csv")
test2 = read_data("Watermelon-test2.csv")

# 基本要求

## 构建ID3决策树

### 计算信息熵

In [3]:
# 计算信息熵
def get_entropy(data):
    # 使用 Counter 统计数据集中每个类别的出现次数
    class_counts = Counter(item[-1] for item in data)

    # 获取数据集总样本数
    total_instances = len(data)

    # 计算信息熵
    entropy = -sum((count/total_instances) * math.log2(count/total_instances) for count in class_counts.values())

    # 返回信息熵和每个类别的频数字典
    return entropy, dict(class_counts)


# 将数据按照某一种类的属性进行分类，并将该行属性删除
def split(data, index, kind):
    ls = [temp[0: index] + temp[index + 1:] for temp in data if temp[index] == kind]
    return ls


### 计算信息增益，选取最佳特征

In [4]:
def get_id3_best_feature(data):
    base_entropy, _ = get_entropy(data)  # 原始的信息熵
    best_info_gain = 0
    best_index = -1

    for i in range(len(data[0]) - 1):
        # 抽取该列数据的所有信息
        feature_values = [index[i] for index in data]
        unique_values = set(feature_values)

        # 计算该列的信息增益
        temp_entropy = 0
        for value in unique_values:
            subset = split(data, i, value)
            probability = len(subset) / float(len(data))
            entropy, _ = get_entropy(subset)
            temp_entropy += probability * entropy

        info_gain = base_entropy - temp_entropy

        # 根据信息增益挑选
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_index = i

    # 返回信息增益最大的特征
    return best_index


### 构建ID3决策树


In [5]:
def get_id3(data, labels):
    # 获取数据集中所有样本的类别列表
    type_list = [index[-1] for index in data]  
    # 计算数据集的信息熵和各个类别的数量
    _, type_count = get_entropy(data)  

    # 如果数据集中只有一种类别，则直接返回该类别
    if len(type_count) == 1: 
        return type_list[0]

    # 获取最佳划分特征的索引
    best_index = get_id3_best_feature(data) 
    best_label = labels[best_index]
    # 创建决策树的根节点
    tree = {best_label: {}}

    # 移除已经使用过的特征标签
    remaining_labels = labels[:best_index] + labels[best_index + 1:]

    # 获取最佳划分特征的所有取值
    feature_values = [example[best_index] for example in data]
    unique_values = set(feature_values) 
    
    # 遍历最佳划分特征的所有取值，递归构建决策树
    for value in unique_values:
        remaining_labels_copy = remaining_labels[:]
        # 递归调用get_id3函数构建子树
        tree[best_label][value] = get_id3(split(data, best_index, value), remaining_labels_copy)

    return tree


### 对测试集进行预测

In [6]:
def id3_predict(testdata, tree, labels):
    # 获取决策树的根节点特征
    firstStr = list(tree.keys())[0]
    # 获取根节点特征对应的子树
    secondDict = tree[firstStr]
    # 获取根节点特征在特征标签中的索引
    featIndex = labels.index(firstStr)
    # 初始化结果变量为空字符串
    result = ''
    
    # 遍历子树的所有分支
    for key in secondDict: 
        # 判断测试数据在根节点特征的取值是否与当前分支相符
        if testdata[featIndex] == key:
            # 判断当前分支是否为叶子节点
            if isinstance(secondDict[key], dict):  # 该分支不是叶子节点，递归
                # 递归调用，继续在子树中查找
                result = id3_predict(testdata, secondDict[key], labels)
            else:
                # 当前分支是叶子节点，直接获取预测结果
                result = secondDict[key]
    # 返回最终的预测结果
    return result

# 计算准确率
def calculate_accuracy(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i][-1] == predicted[i]:
            correct += 1
    accuracy = correct / len(actual)
    return accuracy

In [7]:
labels1 = ['色泽', '根蒂', '敲声', '纹理', '好瓜']
result1 = []
id3_tree = get_id3(train1, labels1)
# print(id3_tree)
for index in test1:
    result1.append(id3_predict(index, id3_tree, labels1))

accuracy = calculate_accuracy(test1, result1)
print("ID3 Accuracy on test1: ", accuracy)

ID3 Accuracy on test1:  0.7


# 中级要求

## 构建C4.5决策树

### 划分数据集

In [8]:
def split_data(data, index, kind, method):
    if method == 0:
        # 如果方法为0，将数据集中属性值小于等于kind的样本划分到左子集
        return [temp[:index] + temp[index + 1:] for temp in data if temp[index] <= kind]
    elif method == 1:
        # 如果方法为1，将数据集中属性值大于kind的样本划分到右子集
        return [temp[:index] + temp[index + 1:] for temp in data if temp[index] > kind]

### 选取最佳特征

In [9]:
def get_best_feature_c45(data):
    base, _ = get_entropy(data)  # 原始信息熵
    info = []

    for j in range(len(data[0]) - 1):
        dic = {}
        for i in data:
            current = i[j]  # 提取结果
            if current not in dic.keys():
                dic[current] = 1  # 创建一个新的类别
            else:
                dic[current] += 1  # 原有类别计数加1

        result = 0
        for key in dic:
            prob = float(dic[key]) / len(data)
            result -= prob * math.log(prob, 2)

        info.append(result)

    best = 0
    best_index = -1
    best_part_value = None  # 如果是离散值，使用该值进行分割

    for i in range(len(data[0]) - 1):
        # 提取该列数据的所有信息
        ls = [index[i] for index in data]
        feature_set = set(ls)

        temp = 0.0
        if type(ls[0]) == type("a"):  # 判断是否为离散值
            for value in feature_set:
                datatemp = split(data, i, value)
                prob = len(datatemp) / float(len(data))
                t, _ = get_entropy(datatemp)
                temp += prob * t
        else:
            ls.sort()
            min_entropy = float("inf")

            for j in range(len(ls) - 1):
                part = (ls[j + 1] + ls[j]) / 2  # 计算划分点
                left = split_data(data, i, part, 0)
                right = split_data(data, i, part, 1)

                temp1, _ = get_entropy(left)
                temp2, _ = get_entropy(right)
                temp = len(left) / len(data) * temp1 + len(right) / len(data) * temp2

                if temp < min_entropy:
                    min_entropy = temp
                    best_part_value = part

            temp = min_entropy

        info_gain = base - temp

        # 根据信息增益挑选
        if info[i] != 0:
            if info_gain / info[i] >= best:
                best = info_gain / info[i]
                best_index = i

    return best_index, best_part_value


### 构建C4.5决策树

In [10]:
def get_c45(data, labels):
    # 获取数据集中最后一列（类别列）的值列表
    typelist = [index[-1] for index in data]
    
    # 计算数据集各个类别的数量
    _, typecount = get_entropy(data)
    
    # 如果只有一个类别，直接返回该类别
    if typecount == len(typelist):
        return typelist[0]
    
    # 获取最优划分属性的索引和划分点
    best_index, part = get_best_feature_c45(data)
    
    # 如果找不到最优属性，返回默认值
    if best_index == -1:
        return "是"
    
    # 判断最优属性是否为离散值
    if isinstance(data[0][best_index], str):
        # 离散值情况下的处理
        bestlabel = labels[best_index]
        Tree = {bestlabel: {}}
        temp = copy.copy(labels)
        feature = [example[best_index] for example in data]
        # 已经选择的特征不再参与分类，将该类别删除
        del temp[best_index]
        # 该属性所有可能取值
        unique_values = set(feature)
        for value in unique_values:
            # 拷贝temp，防止递归影响上层
            s = temp[:]
            # 递归构建子树
            Tree[bestlabel][value] = get_c45(split(data, best_index, value), s)
    else:
        # 连续值情况下的处理
        bestlabel = labels[best_index] + "<" + str(part)
        Tree = {bestlabel: {}}
        temp = labels[:]
        # 已经选择的特征不再参与分类，将该类别删除
        del temp[best_index]
        # 根据划分点划分数据集
        left_data = split_data(data, best_index, part, 0)
        right_data = split_data(data, best_index, part, 1)
        # 递归构建左右子树
        Tree[bestlabel]["是"] = get_c45(left_data, temp)
        Tree[bestlabel]["否"] = get_c45(right_data, temp)
    
    return Tree

### 对测试集进行预测

In [11]:
def c45_predict(data, tree, labels):
    # 获取根节点信息
    root_attribute = list(tree.keys())[0]
    first_label = root_attribute.split('<')[0].strip()  # 提取属性名称

    # 获取树的分支
    branch_dict = tree[root_attribute]
    feat_index = labels.index(first_label)  # 获取属性在数据中的索引

    for key, value in branch_dict.items():
        # 处理离散型特征
        if isinstance(data[feat_index], str):
            if data[feat_index] == key:
                if isinstance(value, dict):  # 如果不是叶节点则递归
                    return c45_predict(data, value, labels)
                else:  # 如果是叶节点则返回结果
                    return value
        else:
            # 处理连续型特征
            split_value = float(root_attribute.split('<')[1])  # 提取划分值
            branch = '是' if data[feat_index] <= split_value else '否'

            if type(branch_dict[branch]).__name__ == 'dict':  # 不是叶节点则递归
                return c45_predict(data, branch_dict[branch], labels)
            else:  # 是叶节点返回结果
                return branch_dict[branch]


In [12]:
labels2 = ["色泽", "根蒂", "敲声", "纹理", "密度", "好瓜"]
result2 = []
c45_tree = get_c45(train2, labels2)
print(c45_tree)
for index in test2:
    result2.append(c45_predict(index, c45_tree, labels2))

accuracy = calculate_accuracy(test2, result2)
print("C4.5 Accuracy on test2: ", accuracy)

{'纹理': {'稍糊': {'敲声': {'沉闷': {'密度<0.6615': {'是': '是', '否': {'根蒂': {'蜷缩': '是', '稍蜷': '是'}}}}, '浊响': {'密度<0.56': {'是': '是', '否': '是'}}}}, '模糊': {'密度<0.29400000000000004': {'是': '是', '否': '是'}}, '清晰': {'根蒂': {'蜷缩': {'密度<0.5820000000000001': {'是': '是', '否': {'敲声': {'沉闷': {'色泽': {'青绿': '是', '乌黑': '是'}}, '浊响': {'色泽': {'青绿': '是', '乌黑': '是'}}}}}}, '稍蜷': {'密度<0.3815': {'是': '是', '否': {'色泽': {'青绿': '是', '乌黑': '是'}}}}, '硬挺': '是'}}}}
C4.5 Accuracy on test2:  0.6


# 高级要求

## 决策树预剪枝

以最大树深为指标，对决策树进行预剪枝
也尝试过使用剪枝后的准确率作为指标对决策树进行后剪枝，但是在两棵树上都没有什么提升 ~~(也可能是我写的有问题)~~

In [13]:
def get_id3_pre(data, labels, max_depth=None):
    # 获取数据集中所有样本的类别列表
    type_list = [index[-1] for index in data]  
    # 计算数据集的信息熵和各个类别的数量
    _, type_count = get_entropy(data)  

    # 如果数据集中只有一种类别，则直接返回该类别
    if len(type_count) == 1: 
        return type_list[0]
    
    # 如果达到最大深度，直接返回出现最频繁的标签
    if max_depth is not None and max_depth == 0:
        return max(type_list, key=type_list.count)

    # 获取最佳划分特征的索引
    best_index = get_id3_best_feature(data) 
    best_label = labels[best_index]
    # 创建决策树的根节点
    tree = {best_label: {}}

    # 移除已经使用过的特征标签
    remaining_labels = labels[:best_index] + labels[best_index + 1:]

    # 获取最佳划分特征的所有取值
    feature_values = [example[best_index] for example in data]
    unique_values = set(feature_values) 
    
    # 遍历最佳划分特征的所有取值，递归构建决策树
    for value in unique_values:
        remaining_labels_copy = remaining_labels[:]
        # 递归调用get_id3函数构建子树
        tree[best_label][value] = get_id3_pre(
            split(data, best_index, value), 
            remaining_labels_copy, 
            max_depth = max_depth - 1 if max_depth is not None else None
        )

    return tree

In [14]:
labels1 = ['色泽', '根蒂', '敲声', '纹理', '好瓜']

max_deep = 0
max_acc = 0

for i in range(1, 10):
    result1 = []
    id3_tree = get_id3_pre(train1, labels1, i)
    for index in test1:
        result1.append(id3_predict(index, id3_tree, labels1))

    accuracy = calculate_accuracy(test1, result1)
    if accuracy > max_acc:
        max_acc = accuracy
        max_deep = i

result1 = []
id3_tree = get_id3_pre(train1, labels1, max_deep)
print(id3_tree)
for index in test1:
    result1.append(id3_predict(index, id3_tree, labels1))

accuracy = calculate_accuracy(test1, result1)
print(f"Max Deep = {max_deep}, Max ID3 Accuracy on test1 = {accuracy}")

{'纹理': {'稍糊': '否', '模糊': '否', '清晰': '是'}}
Max Deep = 1, Max ID3 Accuracy on test1 = 0.8


对于test1数据集，最大树深为1时，准确率最高，为0.8

In [15]:
def get_c45_pre(data, labels, max_depth=None):
    # 获取数据集中最后一列（类别列）的值列表
    typelist = [index[-1] for index in data]
    
    # 计算数据集各个类别的数量
    _, typecount = get_entropy(data)
    
    # 如果只有一个类别，直接返回该类别
    if typecount == len(typelist):
        return typelist[0]
    
    # 如果达到最大深度，直接返回出现最频繁的标签
    if max_depth is not None and max_depth == 0:
        return max(typelist, key=typelist.count)
    
    # 获取最优划分属性的索引和划分点
    best_index, part = get_best_feature_c45(data)
    
    # 如果找不到最优属性，返回默认值
    if best_index == -1:
        return "是"
    
    # 判断最优属性是否为离散值
    if isinstance(data[0][best_index], str):
        # 离散值情况下的处理
        bestlabel = labels[best_index]
        Tree = {bestlabel: {}}
        temp = copy.copy(labels)
        feature = [example[best_index] for example in data]
        # 已经选择的特征不再参与分类，将该类别删除
        del temp[best_index]
        # 该属性所有可能取值
        unique_values = set(feature)
        for value in unique_values:
            # 拷贝temp，防止递归影响上层
            s = temp[:]
            # 递归构建子树
            Tree[bestlabel][value] = get_c45_pre(split(data, best_index, value), s, max_depth=max_depth - 1 if max_depth is not None else None)
    else:
        # 连续值情况下的处理
        bestlabel = labels[best_index] + "<" + str(part)
        Tree = {bestlabel: {}}
        temp = labels[:]
        # 已经选择的特征不再参与分类，将该类别删除
        del temp[best_index]
        # 根据划分点划分数据集
        left_data = split_data(data, best_index, part, 0)
        right_data = split_data(data, best_index, part, 1)
        # 递归构建左右子树
        Tree[bestlabel]["是"] = get_c45_pre(left_data, temp, max_depth=max_depth - 1 if max_depth is not None else None)
        Tree[bestlabel]["否"] = get_c45_pre(right_data, temp, max_depth=max_depth - 1 if max_depth is not None else None)
    
    return Tree

In [16]:
labels2 = ["色泽", "根蒂", "敲声", "纹理", "密度", "好瓜"]

max_deep = 0
max_acc = 0

for i in range(1, 20):
    result2 = []
    c45_tree = get_c45_pre(train2, labels2, i)
    for index in test2:
        result2.append(c45_predict(index, c45_tree, labels2))

    accuracy = calculate_accuracy(test2, result2)
    if accuracy > max_acc:
        max_acc = accuracy
        max_deep = i

result2 = []
c45_tree = get_c45_pre(train2, labels2, i)
for index in test2:
    result2.append(c45_predict(index, c45_tree, labels2))

accuracy = calculate_accuracy(test2, result2)
print(c45_tree)
print(f"Max Deep = {max_deep}, Max C4.5 Accuracy on test2 = {accuracy}")

{'纹理': {'稍糊': {'敲声': {'沉闷': {'密度<0.6615': {'是': '是', '否': {'根蒂': {'蜷缩': '是', '稍蜷': '是'}}}}, '浊响': {'密度<0.56': {'是': '是', '否': '是'}}}}, '模糊': {'密度<0.29400000000000004': {'是': '是', '否': '是'}}, '清晰': {'根蒂': {'蜷缩': {'密度<0.5820000000000001': {'是': '是', '否': {'敲声': {'沉闷': {'色泽': {'青绿': '是', '乌黑': '是'}}, '浊响': {'色泽': {'青绿': '是', '乌黑': '是'}}}}}}, '稍蜷': {'密度<0.3815': {'是': '是', '否': {'色泽': {'青绿': '是', '乌黑': '是'}}}}, '硬挺': '是'}}}}
Max Deep = 2, Max C4.5 Accuracy on test2 = 0.6


对于test2数据集，最大树深为2时，准确率最高，为0.6(树深增加也还是这个值)